In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("../data/authors/authors_data_science.csv")

In [3]:
import os
import sys

current_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.insert(0, parent_dir)

from orcid_data_utilities import retrive_authors_metadata_name

ModuleNotFoundError: No module named 'orcid_data_utilities'

In [3]:
import requests

def search_orcid_bicocca(given_name, family_name):
    base_url = "https://pub.orcid.org/v3.0/expanded-search/"
    query = f"given-names:{given_name} AND family-name:{family_name}"

    headers = {
        "Accept": "application/json"
    }

    params = {
        "q": query
    }

    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Errore nella richiesta: {response.status_code}")
        return None

    results = response.json().get("expanded-result", [])

    if not results:
        print("Nessun risultato trovato.")
        return None

    for res in results:
        affiliations = res.get("institution-name", [])
        if any("bicocca" in aff.lower() for aff in affiliations):
            full_name = res.get("given-names", "") + " " + res.get("family-name", "")
            orcid = res.get("orcid-id", "N/A")
            print(f"Trovato: {full_name} -> ORCID: {orcid}")
            return orcid

    print("Nessun ORCID trovato associato alla Bicocca.")
    return None

In [5]:
def search_orcid_all_matches(given_name, family_name):
    base_url = "https://pub.orcid.org/v3.0/expanded-search/"
    query = f"given-names:{given_name} AND family-name:{family_name}"

    headers = {
        "Accept": "application/json"
    }

    params = {
        "q": query
    }

    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Errore nella richiesta: {response.status_code}")
        return None

    results = response.json().get("expanded-result", [])

    if not results:
        print("Nessun risultato trovato.")
        return None

    # Stampa tutti i match trovati
    for res in results:
        full_name = res.get("given-names", "") + " " + res.get("family-name", "")
        orcid = res.get("orcid-id", "N/A")
        institutions = res.get("institution-name", [])
        print(f"{full_name} -> ORCID: {orcid} | Institutions: {institutions}")

    return orcid

In [9]:
for index, row in df.iterrows():
        family_name = row['Last name']
        given_name = row['Name']
        orcid = search_orcid_bicocca(given_name, family_name)
        if not orcid:
                orcid = search_orcid_all_matches(given_name, family_name)
        df.loc[index, "orcid"] = orcid

Nessun risultato trovato.
Nessun risultato trovato.
Nessun ORCID trovato associato alla Bicocca.
DAVIDE PAOLO  -> ORCID: 0000-0001-8771-5428 | Institutions: []
Paolo  -> ORCID: 0000-0001-6043-3367 | Institutions: ['Fondazione IRCCS Policlinico San Matteo', "UNIVERSITA' DEGLI STUDI DI PAVIA"]
Trovato: Simone  -> ORCID: 0000-0002-7070-1545
Trovato: Pier Giovanni  -> ORCID: 0000-0003-3769-6649
Nessun ORCID trovato associato alla Bicocca.
Mirko  -> ORCID: 0000-0001-9601-0403 | Institutions: []
Trovato: Davide  -> ORCID: 0000-0001-9655-7142
Trovato: Michele  -> ORCID: 0000-0002-2480-966X
Trovato: Gianluca  -> ORCID: 0000-0001-5584-3089
Nessun ORCID trovato associato alla Bicocca.
Nico  -> ORCID: 0009-0004-3430-8326 | Institutions: ['University of Brescia']
Nico  -> ORCID: 0009-0001-4949-0624 | Institutions: ["University of L'Aquila"]
Nessun ORCID trovato associato alla Bicocca.
MARCO  -> ORCID: 0000-0002-6624-8637 | Institutions: []
Nessun ORCID trovato associato alla Bicocca.
Claudio  -> O

In [10]:
df[df["orcid"].isna()]

,Last name,Name,SSD,orcid
0,Andreotti,Alberta Argia,SPS/09,None
1,Bernasconi,Davide Paolo,MED/01,None
4,Cesarini,Mirko,ING-INF/05,None
8,Di Domenica,Nico,SECS-P/08,None
9,Fattore,Marco,SECS-S/03,None
10,Ferretti,Claudio,INF/01,None
11,Fersini,Elisabetta,INF/01,None
17,Messina,Vincenzina,MAT/09,None
18,Monti,Gianna Serafina,SECS-S/01,None
27,Presto,Luca,FIS/07,None


In [10]:
df

,Last name,Name,SSD,orcid
0,Andreotti,Alberta Argia,SPS/09,None
1,Bernasconi,Davide Paolo,MED/01,0000-0001-6043-3367
2,Bianco,Simone,INF/01,0000-0002-7070-1545
3,Bissiri,Pier Giovanni,SECS-S/01,0000-0003-3769-6649
4,Cesarini,Mirko,ING-INF/05,0000-0001-9601-0403
5,Chicco,Davide,INF/01,0000-0001-9655-7142
6,Ciavotta,Michele,INF/01,0000-0002-2480-966X
7,Della Vedova,Gianluca,INF/01,0000-0001-5584-3089
8,Di Domenica,Nico,SECS-P/08,0009-0001-4949-0624
9,Fattore,Marco,SECS-S/03,0000-0002-6624-8637
